In [1]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [3]:
df = pd.read_csv('data/cannabis.csv')

In [4]:
df = df.replace('None', np.nan)
df = df.dropna().reset_index()

In [5]:
df['Flavor']= df['Flavor'].str.replace(',',' ')
df['Effects'] = df['Effects'].str.replace(',', ' ')

In [6]:
df['combined'] = df["Effects"] +  df["Flavor"] + df['Description']

In [7]:

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                       ngram_range = (1,2),
                       max_features = 2000)

# Create a vocabulary and tf-idf score per document
dtm = tfidf.fit_transform(df['combined'])
                         

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)

In [8]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl2'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances

In [9]:
# Loading the saved model
model_pkl2 = open(pickle_filename, 'rb')
NN_model2 = pickle.load(model_pkl2)
print ("Loaded model :: ", NN_model2)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [10]:
# Dump the trained classifier (tfidf)  with Pickle
pickle_filename_1 = 'tfidf.pkl2'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [11]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model2 = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model2)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [12]:
import json
def recommend(user_input):
    temp_df = NN_model2.kneighbors(tfidf_model2.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = df.loc[temp_df[0][i]]['Strain']
        info_effects = df.loc[temp_df[0][i]]['Effects']
        info_flavor = df.loc[temp_df[0][i]]['Flavor']
        info_description = df.loc[temp_df[0][i]]['Description']
        info_rating = df.loc[temp_df[0][i]]['Rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info)  #for engineeers, the return does not work in jupyter lab.  Should work in vsCode.
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dump(info_description)
        #return json.dumps(info_rating)

In [13]:
recommend('hybrid-like in its balanced calm and moderate cerebral effects')

"Jackalope"
"Happy Relaxed Aroused Sleepy Talkative"
"Citrus Earthy Flowery"
"Jackalope is a mostly sativa strain that crosses Chocolope and Black Jack in a flavorful\u00a0combination of berry and chocolate confection. Jack Herer ancestry reveals itself\u00a0through this strain's energized and happy effects, which slowly taper\u00a0down to\u00a0mellow relaxation over time.\u00a0"
4.2
"Denver-Maple"
"Relaxed Sleepy Giggly Tingly Uplifted"
"Sweet Earthy Pine"
"Denver Maple is an indica strain that took 3rd place in the 2014 Denver Cannabis Cup."
4.4
"Belladonna"
"Happy Sleepy Giggly Euphoric Uplifted"
"Peach Earthy Butter"
"Belladonna is a mostly sativa strain that also carries good indica characteristics making for a very nice hybrid. \u00a0A faster flowering time than typical sativa strains makes Belladonna a favorite among growers. \u00a0With a fruity smell and a trippy high Paradise Seeds has bred a winner."
4.0
"Enigma"
"Relaxed Happy Sleepy Giggly Euphoric"
"Citrus Pungent Flowery"